In [1]:
#import standard libraries
import pandas as pd
import numpy as np

#import urllib and beautifulsoup for webcrawler
import urllib.request
from bs4 import BeautifulSoup

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [2]:
#define the website to scrape 
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = urllib.request.urlopen(url)

In [3]:
#load the html
soup = BeautifulSoup(page, "lxml")

#locate the table we want to retrieve 
tables = soup.find_all("table")
table = soup.find('table', class_ = 'wikitable sortable')

In [4]:
#define empty arrays
A = []
B = []
C = []

#loop through table and store columns in arrays
for row in table.findAll('tr'):
    cells = row.findAll('td')
    if len(cells) == 3:
        A.append(cells[0].find(text=True))
        B.append(cells[1].find(text=True))
        C.append(cells[2].find(text=True))

#create dataframe and transfer data        
df = pd.DataFrame(A,columns=['Postal Code'])
df['Borough'] = B
df['Neighborhood'] = C

In [5]:
#drop rows with columns 'not assigned'
missing = df[df['Borough']=='Not assigned\n'].index
missing
df.drop(missing, inplace = True)

In [12]:
#reset the indices 
df.reset_index(drop=True,inplace = True)
df

,Postal Code,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods\n
1,M4A\n,North York\n,Victoria Village\n
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"
...,...,...,...
98,M8X\n,Etobicoke\n,"The Kingsway, Montgomery Road, Old Mill North\n"
99,M4Y\n,Downtown Toronto\n,Church and Wellesley\n
100,M7Y\n,East Toronto\n,"Business reply mail Processing Centre, South C..."
101,M8Y\n,Etobicoke\n,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [6]:
#check for the unique number of postal codes 
len(df['Postal Code'].unique())

103

In [7]:
#check the shape of the dataframe to see if there are any duplicate area codes
df.shape

(103, 3)

In [8]:
#load in the neighborhood coordinate data
coords = pd.read_csv('http://cocl.us/Geospatial_data')
coords.shape

(103, 3)

In [9]:
df['Postal Code'] = df['Postal Code'].str.strip()
coords['Postal Code'] = coords['Postal Code'].str.strip()
mdf = pd.merge(df,coords, on='Postal Code')
mdf

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York\n,Parkwoods\n,43.753259,-79.329656
1,M4A,North York\n,Victoria Village\n,43.725882,-79.315572
2,M5A,Downtown Toronto\n,"Regent Park, Harbourfront\n",43.654260,-79.360636
3,M6A,North York\n,"Lawrence Manor, Lawrence Heights\n",43.718518,-79.464763
4,M7A,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke\n,"The Kingsway, Montgomery Road, Old Mill North\n",43.653654,-79.506944
99,M4Y,Downtown Toronto\n,Church and Wellesley\n,43.665860,-79.383160
100,M7Y,East Toronto\n,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke\n,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
